In [11]:
# 展示文件名进行测试

import os
import SimpleITK as sitk
import matplotlib.pyplot as plt

folder_path = "/media/tianyu/fanty/output_slices/xy/LABEL"

filenames = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

print(filenames)

if len(filenames) >= 300:

    file_to_display = filenames[259]
    
    print(file_to_display)

    file_path = os.path.join(folder_path, file_to_display)

#     img = sitk.ReadImage(file_path)
    
#     img_arr = sitk.GetArrayFromImage(img)
#     plt.imshow(img_arr, cmap='gray')
#     plt.show()
else:
    print("fail")

['MASK_000_0.txt', 'MASK_000_1.txt', 'MASK_000_10.txt', 'MASK_000_100.txt', 'MASK_000_101.txt', 'MASK_000_102.txt', 'MASK_000_103.txt', 'MASK_000_104.txt', 'MASK_000_105.txt', 'MASK_000_106.txt', 'MASK_000_107.txt', 'MASK_000_108.txt', 'MASK_000_109.txt', 'MASK_000_11.txt', 'MASK_000_110.txt', 'MASK_000_111.txt', 'MASK_000_112.txt', 'MASK_000_240.txt', 'MASK_000_241.txt', 'MASK_000_242.txt', 'MASK_000_243.txt', 'MASK_000_244.txt', 'MASK_000_245.txt', 'MASK_000_246.txt', 'MASK_000_247.txt', 'MASK_000_248.txt', 'MASK_000_249.txt', 'MASK_000_25.txt', 'MASK_000_250.txt', 'MASK_000_251.txt', 'MASK_000_252.txt', 'MASK_000_253.txt', 'MASK_000_254.txt', 'MASK_000_255.txt', 'MASK_000_41.txt', 'MASK_000_42.txt', 'MASK_000_43.txt', 'MASK_000_44.txt', 'MASK_000_45.txt', 'MASK_000_46.txt', 'MASK_000_47.txt', 'MASK_000_48.txt', 'MASK_000_49.txt', 'MASK_000_5.txt', 'MASK_000_50.txt', 'MASK_000_51.txt', 'MASK_000_52.txt', 'MASK_000_53.txt', 'MASK_000_54.txt', 'MASK_000_55.txt', 'MASK_000_56.txt', 'MAS

In [13]:
# 批量改名函数

import os

def modify_filename(filename):
    new_filename = filename.replace("MASK", "DATA")

    base, ext = os.path.splitext(new_filename)

    parts = base.split("_")

    parts[-1] = str(int(parts[-1])).zfill(3)

    new_filename = "_".join(parts) + ext

    return new_filename


In [27]:
# 应用 txt yolo数据 2d data文件 产生coco数据及所需的 .json文件

import os
import json
import SimpleITK as sitk
from pycocotools import mask
import cv2
import numpy as np

def parse_txt_file(txt_file_path):
    instances = []
    with open(txt_file_path, 'r') as file:
        lines = file.readlines()
        if not lines:  
            return None
        for line in lines:
            parts = line.strip().split()
            class_index = int(parts[0])
            points = list(map(float, parts[1:]))
            points = [(points[i], points[i + 1]) for i in range(0, len(points), 2)]
            instances.append({
                "class_index": class_index,
                "points": points,
            })
    return instances

def convert(o):
    if isinstance(o, np.generic):
        return o.item()  
    raise TypeError

def generate_coco_json(txt_folder, nrrd_folder, output_json_path):
    data_dict = {
        "images": [],
        "annotations": [],
        "categories": [
            {"id": 0, "name": "benign tumor"},
            {"id": 1, "name": "malignant tumor"}
        ]
    }
    
    txt_files = sorted(os.listdir(txt_folder))
    num_train = int(0.8 * len(txt_files))
    
    for image_id, txt_file in enumerate(txt_files):
        txt_path = os.path.join(txt_folder, txt_file)
        instances = parse_txt_file(txt_path)
        
        txt_file = modify_filename(txt_file)
        nrrd_file = txt_file.replace('.txt', '.nrrd')
        nrrd_path = os.path.join(nrrd_folder, nrrd_file)
        nrrd_data = sitk.GetArrayFromImage(sitk.ReadImage(nrrd_path))
        
        height, width = nrrd_data.shape[:2]
        
        data_dict["images"].append({
            "file_name": txt_file.replace('.txt', '.jpg'),
            "height": height,
            "width": width,
            "id": image_id
        })
        
        if instances:  
            for instance in instances:

                mask_image = np.zeros((height, width))
                cv2.fillPoly(mask_image, [np.array(instance["points"], dtype=np.int32)], 1)
                mask_image = mask_image.astype(np.uint8)
                
                pos = np.where(mask_image == 1)
                xmin = np.min(pos[1])
                xmax = np.max(pos[1])
                ymin = np.min(pos[0])
                ymax = np.max(pos[0])
                
                data_dict["annotations"].append({
                    "id": len(data_dict["annotations"]),
                    "image_id": image_id,
                    "category_id": instance["class_index"],
                    "bbox": [xmin, ymin, xmax-xmin, ymax-ymin],
                    "area": (xmax-xmin)*(ymax-ymin),
                    "segmentation": [point for sublist in instance["points"] for point in sublist],
                    "iscrowd": 0
                })
        
    train_dict = data_dict.copy()
    train_dict["images"] = [img for img in train_dict["images"] if img["id"] < num_train]
    train_dict["annotations"] = [ann for ann in train_dict["annotations"] if ann["image_id"] < num_train]
    
    val_dict = data_dict.copy()
    val_dict["images"] = [img for img in val_dict["images"] if img["id"] >= num_train]
    val_dict["annotations"] = [ann for ann in val_dict["annotations"] if ann["image_id"] >= num_train]
    
    with open(output_json_path + '_train.json', 'w') as f:
        json.dump(train_dict, f, default=convert)
        
    with open(output_json_path + '_val.json', 'w') as f:
        json.dump(val_dict, f, default=convert)
        
generate_coco_json(txt_folder="/media/tianyu/fanty/output_slices/xy/LABEL", nrrd_folder="/media/tianyu/fanty/output_slices/xy/DATA", output_json_path="/media/tianyu/fanty/output_slices/xy/JSON")
